In [8]:
import simpy
import random
import numpy as np

In [11]:
#params for simulation
#num_applications = int(input("Enter the number of applications you want to test: "))
num_employees = int(input("Enter the number of employees you want to test: "))
avg_support_time = 5
avg_customer_interval = 2
sim_time = 120

#counter variable
customers_handled = 0


class CallCenter:
    def __init__(self, env, num_employees, support_time):
        self.env = env
        #number of employees is a resource in this example. If all employees are tied up on a call, then they're
        #not availbe to help a new customer. This is a function in Simpy
        self.staff = simpy.Resource(env, num_employees)
        self.support_time = support_time
        
    def support(self, customer):
        #supply minimum as we don't want to go less than 0
        #if less than 1, take 1, else take the random number generator output for support time
        random_time = max(1, np.random.normal(self.support_time, 4))
        
        #need yield function as this is a generator
        yield self.env.timeout(random_time)
        
        print(f"Support finished for {customer} at {self.env.now:.2f}")
        

def customer(env, name, call_center):
    global customers_handled
    print(f"Customer {name} enters waiting queue at {env.now:.2f}!")
    with call_center.staff.request() as request:
        yield request
        print(f"Customer {name} enters call at {env.now:.2f}")
        yield env.process(call_center.support(name))
        print(f"Customer {name} left call at {env.now:.2f}")
        customers_handled += 1
        
def setup(env, num_empoyees, support_time, avg_customer_interval):
    call_center = CallCenter(env, num_employees, support_time)
    
    for i in range(1, 6):
        env.process(customer(env, i, call_center))
        
    while True:
        yield env.timeout(random.randint(avg_customer_interval - 1, avg_customer_interval + 1))
        i += 1
        env.process(customer(env, i, call_center))
        

print("Starting call center simulation")
env = simpy.Environment()
env.process(setup(env, num_employees, avg_support_time, avg_customer_interval))
env.run(until=sim_time)

print("Customers handled: " + str(customers_handled))

Enter the number of employees you want to test: 6
Starting call center simulation
Customer 1 enters waiting queue at 0.00!
Customer 2 enters waiting queue at 0.00!
Customer 3 enters waiting queue at 0.00!
Customer 4 enters waiting queue at 0.00!
Customer 5 enters waiting queue at 0.00!
Customer 1 enters call at 0.00
Customer 2 enters call at 0.00
Customer 3 enters call at 0.00
Customer 4 enters call at 0.00
Customer 5 enters call at 0.00
Customer 6 enters waiting queue at 1.00!
Customer 6 enters call at 1.00
Support finished for 2 at 1.62
Customer 2 left call at 1.62
Customer 7 enters waiting queue at 2.00!
Customer 7 enters call at 2.00
Support finished for 7 at 3.00
Customer 7 left call at 3.00
Customer 8 enters waiting queue at 4.00!
Customer 8 enters call at 4.00
Support finished for 8 at 5.00
Customer 8 left call at 5.00
Support finished for 5 at 5.04
Customer 5 left call at 5.04
Customer 9 enters waiting queue at 7.00!
Customer 9 enters call at 7.00
Customer 10 enters waiting que